In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import pmdarima as pm
from pmdarima.utils import diff
from sklearn.metrics import mean_squared_error
from pmdarima.metrics import smape
import pickle

In [3]:
df = pd.read_csv('./../data/zri_multifamily_v2.csv')

In [ ]:
df['zip'] = df['zip'].map(lambda x: str(x))

In [ ]:
df['zip'].map(lambda x: len(x)).value_counts()

In [ ]:
df['zip'] = df['zip'].map(lambda x: '0' + x if len(x)<5 else x)

In [ ]:
df['zip'].map(lambda x: len(x)).value_counts()

In [ ]:
df['datetime']=df['datetime'].map(lambda x: datetime.strptime(x, '%Y-%m-%d'))

In [ ]:
df

In [ ]:
# with open('./ts_df.pickle', mode = 'wb') as file:
#     pickle.dump(df, file)
#     file.close()

In [ ]:
train = df[df['year'] <= 2018]

In [ ]:
test = df[df['year'] == 2019]

In [ ]:
ytrain = pd.concat([train['zip'], train['datetime'], train['zri']], axis = 1)

In [ ]:
ytrain['zip'].nunique()

In [ ]:
ytrain

In [ ]:
zips = ytrain['zip'].unique()

In [ ]:
zips

# Trying to make a model for every zip code
Probably don't run the commented cells

In [ ]:
arima = pm.arima.auto_arima

In [ ]:
len(ts_dict['01001'])

In [ ]:
ts_dict['01001']

In [ ]:
# save ts_dict as pickle to avoid runtime of splitting
# with open('./ts_dict.pickle', mode = 'wb') as file:
#     pickle.dump(ts_dict, file)
#     file.close()

In [ ]:
# with open('./ts_dict.pickle', mode = 'rb') as file:
#     asdf = pickle.load(file)

In [ ]:
models_dict = {}

In [ ]:
# start = datetime.now()
# for item in zips:
#     curr_model = arima(ts_dict[item], start_p=0, start_q=0, m=12)
#     models_dict[item] = (curr_model.order, curr_model.seasonal_order)
# elapsed = datetime.now() - start
# print(elapsed)

In [ ]:
len(models_dict)

In [ ]:
len(ts_dict)

In [ ]:
(models_dict)

# Example of one model on zip 27518

Found good guidelines on what (p, d, q) mean and how to interpret them here:
https://towardsdatascience.com/time-series-forecasting-arima-models-7f221e9eee06

In [ ]:
df3 = train[train['zip']=='27518'].loc[:,['zri', 'datetime']]
# filter for a random zip code and make a time series dataframe

In [ ]:
df3

In [ ]:
fig = px.line(df3, x = 'datetime', y = 'zri')
fig
# stationarity is a requirement for arima
# stationary time series = the mean does not change over time
# if there is an overall trend in the time-series you take differences

In [ ]:
pm.plot_acf(df3['zri'], lags = 50, alpha = 0.05)

In [ ]:
pm.plot_pacf(df3['zri'], lags = 20)

In [ ]:
# pmdarima comes with a function to difference a time-series for you
zri_dif = pm.arima.utils.diff(df3['zri'], 1, 1) # lag = 1 and number of differences = 1

In [ ]:
dif1 = df3.copy().iloc[1:,:] # drop 1st row bc first row always becomes nan when differencing

In [ ]:
dif1['zri']=zri_dif

In [ ]:
fig = px.line(dif1, x = 'datetime', y = 'zri')
fig
# now looks approximately stationary centered around 0

In [ ]:
pm.plot_acf(dif1['zri'], lags = 80)

In [ ]:
pm.plot_pacf(dif1['zri'], lags = 20)

In [ ]:
model = pm.arima.auto_arima(df3['zri'], start_p=0, start_q=0, m=12, test = 'kpss')
model

In [ ]:
model.order # model sees that you have to set d=1

In [ ]:
model.summary()

In [ ]:
true = test[(test['zip']=='27518') & (test['year']==2018)]

In [ ]:
def forecast_one_step():
    fc, conf_int = model.predict(n_periods=1, return_conf_int=True)
    return (
        fc.tolist()[0],
        np.asarray(conf_int).tolist()[0])

pred = []
confidence_intervals = []

for new_ob in true['zri']:
    fc, conf = forecast_one_step()
    pred.append(fc)
    confidence_intervals.append(conf)

    # Updates the existing model with the prediction it just made
    model.update(new_ob)

In [ ]:
pred

In [ ]:
fig = px.line(df3, x = 'datetime', y = 'zri')
fig.add_trace(go.Line(x = true['datetime'], y = true['zri'], mode = 'lines'))
fig.add_trace(go.Line(x = true['datetime'], y = pred, mode = 'lines'))
fig.add_trace(go.Line(x = true['datetime'], y = np.asarray(confidence_intervals)[:,0], mode = 'lines'))
fig.add_trace(go.Line(x = true['datetime'], y = np.asarray(confidence_intervals)[:,1], mode = 'lines', fill = 'tonexty'))

In [ ]:
np.asarray(confidence_intervals)

In [ ]:
print(f"Mean squared error: {mean_squared_error(true['zri'], pred)}")
print(f"SMAPE: {smape(true['zri'], pred)}")
# smape of 0 is perfect accuracy, lower smape is better